In [2]:
import numpy as np

where these circles get placed

In [3]:
def generateCircle(numberOfItems, pixelSpace):
    r = 125                                 # An arbitrary value for the radius

    angle = np.linspace(0, 2*np.pi, numberOfItems) # A vector covering all angles from
                                          # 0 to 2*pi (the full circle in radians) 
                                          # with an arbitrary number of 
                                          # elements, 1000 in this example
    X = (np.cos(angle)*pixelSpace).astype(int)
    Y = (np.sin(angle)*pixelSpace).astype(int)

    circlePosCoords = [[x, y] for x, y in zip(X, Y)]
    
    return circlePosCoords

circlePosGrid = generateCircle(numberOfItems=9, pixelSpace=125)[:-1]
circlePosGrid

[[125, 0],
 [88, 88],
 [0, 125],
 [-88, 88],
 [-125, 0],
 [-88, -88],
 [0, -125],
 [88, -88]]

lets say 30 total items in a stream

at least 300 presentation and 700 for response. or 500 ms and 500 ms is fine too.

30 stimuli, 7 targets. 30 second block, 10 second rest. get a response for every one.

3 levels of nback. 1, 2, 3

how many blocks? 10 repetitions of each level.

pseudo-code:

arrayHistory = []

if it is non taget next: get arrayHistory[-level] and attempt to not get it up.
if it is target next: get arrayHistory[-level] and repeat the value. 
    

In [4]:
def createBlockStimLabels(level, possible_stimuli, numItemsPerTrial = 30, targetProbability = .25):
#     level = 2
#     possible_stimuli = circlePosGrid  # Set the possible stimuli
    
    possible_stimuli_indx = np.arange(0, 8, 1)


    totalNumTargets = int(numItemsPerTrial * targetProbability)
#     print(totalNumTargets)

    stimTargetLabeling = np.ones(totalNumTargets)
    stimNonTargetLabeling = np.zeros(numItemsPerTrial - totalNumTargets)

    stimLabels = np.concatenate([stimTargetLabeling, stimNonTargetLabeling[level:]])

#     rng = np.random.default_rng(0) # pseudo-random for participant in case of error.
#     rng.shuffle(stimLabels) 
    
    stimLabels = np.random.permutation(stimLabels)

    stimLabels = np.concatenate([stimNonTargetLabeling[:level] ,stimLabels ])

    arrayHistory = []
    for i in range(len(stimLabels)):

        if i < level:
            random_indx = np.random.choice(possible_stimuli_indx)
#             print(random_indx, possible_stimuli[random_indx])
            arrayHistory.append(possible_stimuli[random_indx])
        elif stimLabels[i] == 0:
#             print('NonTarget')

            while True:
                random_indx = np.random.choice(possible_stimuli_indx)
                
                # if possible_stimuli[random_indx] != arrayHistory[-level]:
                if (possible_stimuli[random_indx] != arrayHistory[-level]).all():
#                     print(random_indx, possible_stimuli[random_indx])
                    arrayHistory.append(possible_stimuli[random_indx])
                    break

        elif  stimLabels[i] == 1:
#             print('Target')
#             print(arrayHistory[-level])
            arrayHistory.append(arrayHistory[-level])
            
            
    return stimLabels, np.asarray(arrayHistory)


def createBlocks(possible_stimuli, levels = [1, 2, 4], numTrials = 10, numItemsPerTrial = 30):
    experimentBlocksStim = np.zeros([len(levels), numTrials, numItemsPerTrial, 2])
    experimentBlocksLabels = np.zeros([len(levels), numTrials, numItemsPerTrial])
    
    for i, level in enumerate(levels):
        for j in range(numTrials):
#             print(level, i)
            stimLabels, stimValues = createBlockStimLabels(level = level, possible_stimuli = possible_stimuli)
            experimentBlocksStim[i, j, :, :] = stimValues
            experimentBlocksLabels[i, j, :] = stimLabels

    return experimentBlocksStim, experimentBlocksLabels
#     stimLabels, stimValues = createBlockStimLabels(level = 4, possible_stimuli = possible_stimuli)
# stimLabels

In [5]:
experimentBlocksStim, experimentBlocksLabels = createBlocks(possible_stimuli = circlePosGrid)

In [6]:
print(experimentBlocksStim.shape, experimentBlocksLabels.shape)

(3, 10, 30, 2) (3, 10, 30)


Test if all looks good here, then save.

In [238]:
for label, coord in zip(experimentBlocksLabels[2, 0, :], experimentBlocksStim[2, 0, :, :]):
    print(label, coord)

0.0 [   0. -125.]
0.0 [-125.    0.]
0.0 [-88. -88.]
0.0 [-125.    0.]
0.0 [  0. 125.]
0.0 [  0. 125.]
0.0 [-125.    0.]
0.0 [88. 88.]
1.0 [  0. 125.]
1.0 [  0. 125.]
0.0 [-88. -88.]
1.0 [88. 88.]
0.0 [125.   0.]
0.0 [-88.  88.]
0.0 [88. 88.]
0.0 [-88. -88.]
0.0 [  0. 125.]
1.0 [-88.  88.]
1.0 [88. 88.]
0.0 [  0. 125.]
0.0 [125.   0.]
0.0 [  0. 125.]
0.0 [   0. -125.]
1.0 [  0. 125.]
0.0 [-88.  88.]
0.0 [125.   0.]
0.0 [-88. -88.]
0.0 [ 88. -88.]
0.0 [  0. 125.]
1.0 [125.   0.]


In [239]:
np.save('participant0_stim.npy', experimentBlocksStim)
np.save('participant0_labels.npy', experimentBlocksLabels)

In [240]:
experimentBlocksStim = np.load('participant0_stim.npy')
experimentBlocksLabels = np.load('participant0_labels.npy')


In [241]:
experimentBlocksStim.shape

(3, 10, 30, 2)

In [242]:
experimentBlocksStim[0, :, :, :].shape

(10, 30, 2)

In [243]:
for trials in experimentBlocksStim[0, :, :, :]:
    print(trials.shape)

(30, 2)
(30, 2)
(30, 2)
(30, 2)
(30, 2)
(30, 2)
(30, 2)
(30, 2)
(30, 2)
(30, 2)


In [7]:
level = 3
numBlocks = 10
numStimuli = 30


for block in range(numBlocks):
    for lvl in range(level):
        for stim in range(numStimuli):
            print(lvl, block, stim)

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 0 5
0 0 6
0 0 7
0 0 8
0 0 9
0 0 10
0 0 11
0 0 12
0 0 13
0 0 14
0 0 15
0 0 16
0 0 17
0 0 18
0 0 19
0 0 20
0 0 21
0 0 22
0 0 23
0 0 24
0 0 25
0 0 26
0 0 27
0 0 28
0 0 29
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 0 5
1 0 6
1 0 7
1 0 8
1 0 9
1 0 10
1 0 11
1 0 12
1 0 13
1 0 14
1 0 15
1 0 16
1 0 17
1 0 18
1 0 19
1 0 20
1 0 21
1 0 22
1 0 23
1 0 24
1 0 25
1 0 26
1 0 27
1 0 28
1 0 29
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
2 0 5
2 0 6
2 0 7
2 0 8
2 0 9
2 0 10
2 0 11
2 0 12
2 0 13
2 0 14
2 0 15
2 0 16
2 0 17
2 0 18
2 0 19
2 0 20
2 0 21
2 0 22
2 0 23
2 0 24
2 0 25
2 0 26
2 0 27
2 0 28
2 0 29
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 1 10
0 1 11
0 1 12
0 1 13
0 1 14
0 1 15
0 1 16
0 1 17
0 1 18
0 1 19
0 1 20
0 1 21
0 1 22
0 1 23
0 1 24
0 1 25
0 1 26
0 1 27
0 1 28
0 1 29
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 1 6
1 1 7
1 1 8
1 1 9
1 1 10
1 1 11
1 1 12
1 1 13
1 1 14
1 1 15
1 1 16
1 1 17
1 1 18
1 1 19
1 1 20
1 1 21
1 1 22
1 1 23
1 1 24
1 1 25
1 1 26
1 1 27
1 1 28
1 1 29
